In [1]:
import numpy as np
import time
import random as rnd
#1. Hashing Task!
#Bloom Filter class
#In this first section of the task we are going to write a Bloom Filter class, i.e. a class representing and implementing everything a bloom filter should be and should do. This will be useful for a more structured as well as a more organized code.

#This bloom filter class will have two instance attributes: Bloom_Filter.array and Bloom_Filter.hash_functions. The first will contain the array representing the bloom filter, the second is a list of hash functions which will be used to insert and search elements on the data structure.

#To represent Bloom_Filter.array we're going to use a numpy array instead of a simple Python list. That's because we'll work with very large arrays and numpy is a lot more efficient both in terms of memory and in terms of computational efficiency.

#The initializer of the class will take two parameters: the size size of the array representing the bloom filter and the list of hash functions hash_function which will be used to work with the data structure. Both these properties are private and immutable so it will not be possible to change the behaviour of a bloom filter once it is initialized with our class.

#In the end the class will have two instance methods: Bloom_Filter.insert(element) and Bloom_Filter.check(element) for inserting and searching items on the bloom filter.

#In [71]:
# This class is going to represent a bloom filter, so that we can organize all the implementation and methods
# of the data structure in a single class.
class Bloom_Filter:
    
    # To the constructor we're going to pass the size of the array representing the bloom filter
    # and the list of hash functions that will be used for our methods
    def _init_(self, size, hash_functions):
        self._array = np.empty(size, dtype = bool)
        self._hash_functions = hash_functions
    
    # This function is for adding elements to the bloom filter
    def insert(self, element):
        for function in self._hash_functions:
            self._array[function(element)] = True
            
    # This function is for checking if an element is possibly on the bloom filter or definitely not in it.
    # It returns True if the element is possibly on it, False if it's definetely not on it.
    def check(self, element):
        for function in self._hash_functions:
            if(not self._array[function(element)]):
                return(False)
        return(True)
#Choosing parameters
#Since we are going to use a bloom filter for our task, one of the first things we have to do is choosing the size $m$ of the array representing it. We know the folllowing approximate formula to get a reasonable value of $m$ given an error tolerance $p$ as well as the size $n$ of the elements we are going to insert on the set:$$
    m = -\frac{n \ln{p}}{(\ln{2})^{2}}
$$.

#For the error tolerance $p$ we are going to choose the value $0.01$ so that we'll have only a $1\%$ rate of false positives.

#To find $n$ we are going to inspect the data we are given. Obviously, since the purpose of the task is to not save all the given passwords in memory, we are going to count the number of passwords without saving all of them in memory. To do this we are just going to open our file of passwords and count how many lines it has (since we already know there's a password per line) by using a counter.

#In [3]:
passwords = open("passwords1.txt", "r")
# It's worth noting that when Python opens a file it's not going to save it in memory
# so we are not cheating on our task by just opening the file if we don't read it all at once

counter = 0
while(passwords.readline()):
    counter = counter + 1
passwords.close()
print(counter)
100000000
We have found out thar our list consists of $100$ milions passwords. Knowing this we can finally compute $m$ with the formula given above and get$$
    m = 958505838
$$.

Having established $p$ and $m$ we only need to find the number of hash functions $k$ we're going to code and use for our task. Again, there's a formula with which we'll get $k$ easily:$$
    k = \frac{m}{n}\ln{2}
$$and we get$$
    k = 6.64
$$so we're going to use seven hash functions.

Hash functions
This is one critical section. Here we are going to code the hash functions we'll use for our bloom filter. As we have established on the previous sections we are going to code seven of these functions and we'll try to make them as good as we can, meaning that not only they need to be good hash functions, but also independent from each other so to increase the efficiency of our data structure.

A good hash function really depends on the distribution of the elements it's going to convert. That's why, before coding our functions, we're going to insepect our passwords data hoping to find some information about its underlying distribution.

We already know that a password is a string of 20 characters, so we want to find what characters are possibly contained in a password as well as if some characters tend to appear more often than others.

In [4]:
passwords = open("passwords1.txt", "r")

# We're going to save the minimum as well the maximum possible character in our file (characters are ordered by their ASCII code)
minimum = 102
maximum = 102

# We're going to look at only the first 1'000'000 entries of the file so to speed up the process
# implicitly assuming that the underlying distribution is homogenous throughout the file
for _ in range(1000000):
    string = passwords.readline()
    for character in string[:19]: # It's important we get rid of the last character, which is always a "\n"
        if(ord(character) < minimum):
            minimum = ord(character)
        if(ord(character) > maximum):
            maximum = ord(character)

print(minimum, chr(minimum))
print(maximum, chr(maximum))
passwords.close()
33 !
122 z
#We see that every password can contain characters ranging from "!" to "z".

#We now want to know if any of this character appears more often than the others.

#In [4]:
passwords = open("passwords1.txt", "r")

# Here we're going to save how many times a character appears on the file, at position i will be the number of times
# chr(i + 33) appeared
counter = [0] * (122 - 33 + 1)

# Again we're just looking at the first 1'000'000 to speed up the process
for _ in range(1000000):
    string = passwords.readline()
    for character in string[:19]:
        counter[ord(character) - 33] += 1

passwords.close()
In [5]:
counter
Out[5]:
[226536,
 226375,
 226357,
 226105,
 226044,
 226000,
 226268,
 226404,
 225767,
 225890,
 225885,
 226388,
 226831,
 225541,
 225986,
 226636,
 225616,
 227077,
 226304,
 227385,
 226377,
 225768,
 226336,
 226474,
 226330,
 226024,
 226416,
 226617,
 226811,
 226216,
 226053,
 226097,
 225798,
 226659,
 225852,
 226279,
 226296,
 226135,
 226755,
 226109,
 226002,
 225869,
 226628,
 225940,
 226091,
 226075,
 225593,
 225928,
 225867,
 226701,
 225958,
 226349,
 226193,
 226762,
 225935,
 226347,
 226287,
 226075,
 0,
 0,
 0,
 0,
 0,
 0,
 226032,
 225611,
 226913,
 226309,
 225951,
 226027,
 225492,
 226486,
 225835,
 225963,
 226979,
 226746,
 225956,
 226440,
 226128,
 225894,
 225349,
 225799,
 226374,
 226345,
 225788,
 225929,
 225759,
 225880,
 226971,
 225647]
Looking at this list we get two useful informations. First of all we see that characters whose ASCII code ranges from 91 ton 96 never appear on the file. Moreover we note that the remaining character appear, approximately, with the same probability in a password.

Based on all these informations we're going to make the following assumption: every password in the file is a randome string where every character is independently drawn in the set of characters whose ASCII code ranges from 33 to 122, excluding those one whose code ranges from 91 to 96.

The efficiency of the algorithms we're going to use will tell us if this has been a reasonable assumption, but for now we stick to it.

First hash function
For our first hash function we're going to read every password as a number in base $84$ (the numbers of possible characters appearing in a password) and then take the remainder of the division between this number and $958505838$. This should be a good hash function for the following reason: based on our previous assumption, a password is a number drawn uniformly from $0$ to $84^{20} - 1$. Given a number $x$ such that $0 \leq x &lt; 958505838$ we get that the probability $p(x)$ of getting $x$ from a random password with this function is approximately $\frac{1}{958505838}$.

For now we code a function to get the base 10 value of a character which will be needed to treat every string as a number in base 84.

In [72]:
def get_base_10(character):
    value = ord(character)
    
    # We remember that values ranging from 91 to 96 do not appear
    if(value < 91):
        return(value - 33)
    else:
        return(value - 39)
We are now going to code our first hash function hash_1(string). Given a string $x_1x_2x_3\cdots x_{20}$ this will represent the number$$
    x_{1} + x_{2} 84 + x_{3} 84^{2} + \cdots + x_{20} 84^{19}
$$and what we need is to take the ramainder between this number and $958505838$. To do this we're going to use Horner's method to compute a polynomial, always working $\text{mod } 958505838$.

In [73]:
def hash_1(string):
    value  = 0
    for index in range(len(string) - 1, -1, -1):
        value = (84 * value + get_base_10(string[index])) % 958505838
    return(value)
Other hash functions
For the other six hash functions we're going to change a little our first function hash_1. In fact we're going to use exactly this function, but instead of using it in our normal string, we're reordering the characters of the string and apply the function to the result.

To start we're going to define a function to change, in some way, the order of characters in a string. We can't change this order casually, since every hash function must be deterministic. What we do is rotating the characters in a string by a step $s$ and this should be enough to obtain a totally different value from the one resulting from hash_1.

In [74]:
def rotate_string(string, step):
    return(string[step:] + string[:step])
We can now define our other six functions and since they will be all similar we're going to define an high order function which will take as parameters one hash function (in our case hash_1) and a number $k$ and will return our $k$-th hash function.

In [75]:
def hash_function(first_hash_function, k):
    
    # Our k-th hash function will just apply hash_1 to the string rotated by k steps
    return(lambda x : hash_1(rotate_string(x, k - 1)))
We now save all our hash functions in a list so that we can pass it as a parameter when we're going to inizialize our Bloom_Filter class.

In [76]:
hash_functions = [hash_function(hash_1, index + 1) for index in range(7)]
As we have already said at the start of the sections, even if our functions seem to be good hash functions by themselves, it's not guaranteed that they are good hash functions for our bloom filter implementation. It could happen that their result are highly dependent, thus decreasing the efficiency of our structure.

We should find a way to test the independence of our hash functions, but unfortunately it's actually impossible that they are exactly independent.

In fact let's suppose for a moment that they are independent. Then taken any seven values $v = (v_{1},\dots, v_{7})$ from $\{0, 1, \dots, 958505838\}$ the probability $p_{(v_{1},\dots, v_{7})}$ of getting $v$ after drawing a random string of $20$ characters subject to our restriction would be$$
    p_{v_{1}}p_{v_{2}}\cdots p_{v_{7}} = \left(\frac{1}{958505838}\right)^{7} \approx \frac{1}{7 \cdot 10^{62}}
$$at the same time it would be$$
    p_{v} = \sum_{s \in S_{v}} p(s)
$$where $S_{v}$ is the set of strings giving $v$ after applying the seven hash functions. Given $s$ we have$$
    p(s) = \left(\frac{1}{84}\right)^{20} \approx \frac{1}{3 \cdot 10^{38}}
$$but this means, by the second equation,$$
    p_{(v_{1},\dots, v_{7})} \geq \frac{1}{3 \cdot 10^{38}}
$$which is totally in contrast with our first equation.

Since we have proved that it's impossible to find seven hash functions exactly independent for our purpose, we're going to use the seven functions we have already coded, hoping they will be enough "random" for a good implementation of the bloom filter. The running time of our algorithm will help us finding out if everything works enough fine later.

The algorithm
We can finally code the algorithm for solving our task. We're going to code the solution in a function for a more organized and structured code.

In [78]:
# The function takes as parameters the name of the file containing the first data set, the name of the file 
# containing the second data set, the size m of the array used to represen the bloom filter and
# the list of hash functions used by the bloom filter

# The function returns the number of strings from the second data set that are possibly contained in the first data set
# and the execution time for finding this number
def task(first_data_set, second_data_set, m, hash_functions):
    
    # We initialize our bloom filter
    bloom_filter = Bloom_Filter(m, hash_functions)
    
    # We add every string in the first data set to the bloom filter
    strings = open(first_data_set, "r")
    start = time.time()
    while(True):
        string = strings.readline()
        if(string == ""):
            break
        string = string[:len(string) - 1] # We need to get rid of the "\n" at the end
        bloom_filter.insert(string)
    strings.close()
    
    # We now check how many strings from the second data set are probably on the first data set
    # and we also create a list containing this possibly duplicates
    strings = open(second_data_set, "r")
    possibly_duplicates = []
    while(True):
        string = strings.readline()
        if(string == ""):
            break
        string = string[:len(string) - 1]
        if(bloom_filter.check(string)):
            possibly_duplicates.append(string)
    end = time.time()
    strings.close()
    
    return((possibly_duplicates, end - start))
We are going to run the function and thus completing our task.

In [80]:
result = task("passwords1.txt", "passwords2.txt", 958505838, hash_functions)

# We print the asked results
print('Number of hash functions used: ', len(hash_functions))
print('Number of possibly duplicates: ', len(result[0]))
print('Probability of false positives: 0.01')
print('Execution time: ', result[1])
Bonus
Here we are going to count the exact number of false positives we got on the previous task. To do so we're going to use a hashing table and the hash function hash_1 in the following way.

We create an array hash_duplicates of size $958505838$ where every entry is a null value or a list of strings based on our list of possibly duplicates. In fact, for every string s in our list, we compute hash_1(s) and add that string to the hash_1(s)-th position of our first array.

We then iterate over the first data set of passwords and for every password p we check if p is in the hash_1(p)-th position of our array. If the answer is yes we're going to remove that string from hash_duplicates[hash_1(p)].

In the end we iterate through hash_duplicates for reamining strings, those will be our false positives.

Assuming that our hash function behaves well (and we hope so based on our discussions in the previous sections) all this process should take an amount of time linear in the size $n = 100000000$ of the first data set, which seems reasonable.

So let's start by creating our array hash_duplicates.

In [44]:
# With this function we create an hash table of size size and with hash function hash_function and immediately
# populate it with some data
def hash_table(size, data, hash_function):
    hash_array = np.empty(size, dtype = list)
    for element in data:
        index = hash_function(element)
        if(hash_array[index] is None):
            hash_array[index] = [element]
        else:
            hash_array[index].append(element)
    return(hash_array)

SyntaxError: invalid syntax (<ipython-input-1-ca5d62b38a32>, line 4)